In [9]:
data_file='D:/Existing Base.csv'

import pandas as pd
import math
from sklearn.cross_validation import train_test_split
from sklearn import tree
import numpy as np
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.metrics import roc_auc_score

%matplotlib inline

bd=pd.read_csv(data_file)

In [10]:
bd.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


We'll be using same set of data, lets reuse the data prep steps used earlier . 

In [11]:
# data prep from previous module
bd.loc[bd["children"]=="Zero","children"]="0"
bd.loc[bd["children"]=="4+","children"]="4"
bd["children"]=pd.to_numeric(bd["children"],errors="coerce")
bd["y"]=1
bd.loc[bd["Revenue Grid"]==2,"y"]=0
bd=bd.drop(["Revenue Grid"],1)
for i in range(len(bd)):
    if bd["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        bd.loc[i,"age_band"]="ab_10"
    if bd["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        bd.loc[i,"age_band"]="ab_11"
    if bd["age_band"][i]=="36-40":
        bd.loc[i,"age_band"]="ab_13"
    if bd["age_band"][i]=="18-21":
        bd.loc[i,"age_band"]="ab_17"
    if bd["age_band"][i]=="61-65":
        bd.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(bd["age_band"])
bd=pd.concat([bd,ab_dummies],1)
bd=bd.drop(["age_band","Unknown"],1)
bd["st_partner"]=0
bd["st_singleNm"]=0
bd["st_divSep"]=0
bd.loc[bd["status"]=="Partner","st_partner"]=1
bd.loc[bd["status"]=="Single/Never Married","st_singleNm"]=1
bd.loc[bd["status"]=="Divorced/Separated","st_divSep"]=1
bd=bd.drop(["status"],1)
for i in range(len(bd)):
    if bd["occupation"][i] in ["Unknown","Student","Secretarial/Admin",
                               "Other","Manual Worker"]:
        bd.loc[i,"occupation"]="oc_11"
    if bd["occupation"][i] in ["Professional","Business Manager"]:
        bd.loc[i,"occupation"]="oc_12"
    if bd["occupation"][i]=="Retired":
        bd.loc[i,"occupation"]="oc_10"
oc_dummies=pd.get_dummies(bd["occupation"])
bd=pd.concat([bd,oc_dummies],1)

bd=bd.drop(["occupation","Housewife"],1)
bd["ocp_10"]=0
bd["ocp_12"]=0

for i in range(len(bd)):
    if bd["occupation_partner"][i] in ["Unknown","Retired","Other"]:
        bd.loc[i,"ocp_10"]=1
    if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
        bd.loc[i,"ocp_12"]=1
bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)
bd["hs_own"]=0
bd.loc[bd["home_status"]=="Own Home","hs_own"]=1
del bd["home_status"]
bd["gender_f"]=0
bd.loc[bd["gender"]=="Female","gender_f"]=1
del bd["gender"]
bd["semp_yes"]=0
bd.loc[bd["self_employed"]=="Yes","semp_yes"]=1
del bd["self_employed"]
bd["semp_part_yes"]=0
bd.loc[bd["self_employed_partner"]=="Yes","semp_part_yes"]=1
del bd["self_employed_partner"]
bd["fi"]=4
bd.loc[bd["family_income"]=="< 8,000, >= 4,000","fi"]=6
bd.loc[bd["family_income"]=="<10,000, >= 8,000","fi"]=9
bd.loc[bd["family_income"]=="<12,500, >=10,000","fi"]=11.25
bd.loc[bd["family_income"]=="<15,000, >=12,500","fi"]=13.75
bd.loc[bd["family_income"]=="<17,500, >=15,000","fi"]=16.25
bd.loc[bd["family_income"]=="<20,000, >=17,500","fi"]=18.75
bd.loc[bd["family_income"]=="<22,500, >=20,000","fi"]=21.25
bd.loc[bd["family_income"]=="<25,000, >=22,500","fi"]=23.75
bd.loc[bd["family_income"]=="<27,500, >=25,000","fi"]=26.25
bd.loc[bd["family_income"]=="<30,000, >=27,500","fi"]=28.75
bd.loc[bd["family_income"]==">=35,000","fi"]=35
bd=bd.drop(["family_income"],1)

We'll remove the missing values first and divide our data into two parts before starting with building the model.

In [12]:
bd_train, bd_test = train_test_split(bd, test_size = 0.2,random_state=2)
x_train=bd_train.drop(["y","REF_NO"],1)
y_train=bd_train["y"]
x_test=bd_test.drop(["y","REF_NO"],1)
y_test=bd_test["y"]
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [38]:
# Standardise data
for i in x_train.columns:
    x_avg = np.mean(x_train[i])
    x_std = np.std(x_train[i])
    x_train[i] = (x_train[i] - x_avg)/ x_std
    x_test[i] = (x_test[i] - x_avg)/ x_std

In [39]:
x_test.head()

,children,year_last_moved,Average Credit Card Transaction,Balance Transfer,Term Deposit,Life Insurance,Medical Insurance,Average A/C Balance,Personal Loan,Investment in Mutual Fund,...,oc_10,oc_11,oc_12,ocp_10,ocp_12,hs_own,gender_f,semp_yes,semp_part_yes,fi
184,0.382662,0.086212,0.205825,0.041508,0.605524,0.232634,-0.357272,0.632129,0.139577,1.184223,...,-0.52722,1.379675,-0.675794,-0.921121,-0.263915,0.284672,0.571096,-0.278305,-0.356923,-0.130839
1546,2.549035,0.080613,0.679507,0.486769,0.933172,1.828413,0.755127,0.969388,0.162329,0.083886,...,-0.52722,1.379675,-0.675794,1.085634,-0.263915,0.284672,0.571096,-0.278305,2.801727,1.268599
9912,1.465848,0.063816,-0.460344,-0.585723,-0.505524,-0.684646,-0.588906,1.295348,-0.299129,-0.645013,...,-0.52722,1.379675,-0.675794,-0.921121,-0.263915,0.284672,0.571096,-0.278305,-0.356923,-0.385282
5078,-0.700525,0.103009,-0.460344,-0.585849,-0.505524,-0.501353,-0.588906,-0.051299,-0.299129,-0.205060,...,-0.52722,-0.724808,1.479741,-0.921121,-0.263915,0.284672,-1.751019,-0.278305,-0.356923,0.632491
7138,2.549035,0.131004,-0.460344,-0.585849,-0.505524,-0.684748,-0.588906,-0.128225,-0.299129,0.417345,...,-0.52722,-0.724808,1.479741,-0.921121,-0.263915,0.284672,0.571096,-0.278305,-0.356923,1.268599


## KNN

In [13]:
import numpy as np

from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [14]:
clf = KNeighborsClassifier()
# this here is the base classifier we are going to try
# we will be supplying different parameter ranges to our randomSearchCV which in turn
# will pass it on to this classifier

# Utility function to report best scores. This simply accepts grid scores from 
# our randomSearchCV/GridSearchCV and picks and gives top few combination according to 
# their scores

def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    # above line selects top n grid scores
    # for loop below , prints the rank, score and parameter combination
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
    
# RandomSearchCV/GridSearchCV accept parameters values as dictionaries.
# In example given below we have constructed dictionary for 
#different parameter values that we want to
# try for randomForest model
param_dist = {'n_neighbors':[5,7],
              'weights':['uniform', 'distance']
              }
# run randomized search
n_iter_search = 4
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                  scoring='roc_auc',
                                  random_state=2)
random_search.fit(x_train, y_train)
# report(random_search.grid_scores_)

Model with rank: 1
Mean validation score: 0.957 (std: 0.014)
Parameters: {'weights': 'distance', 'n_neighbors': 7}

Model with rank: 2
Mean validation score: 0.956 (std: 0.014)
Parameters: {'weights': 'uniform', 'n_neighbors': 7}

Model with rank: 3
Mean validation score: 0.944 (std: 0.017)
Parameters: {'weights': 'distance', 'n_neighbors': 5}



In [ ]:
random_search.best_params_

In [15]:
# knn=KNeighborsClassifier(weights='distance', n_neighbors= 7)
knn = random_search.best_estimator_
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='distance')

In [16]:
predicted=knn.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

predicted     0    1
real                
0          1796   11
1            84  140


In [17]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)
print('Precision is :', TP/(TP + FP))

140 1796 11 84
Accuracy is : 0.953225012309
Sensitivity is : 0.625
Specificity is : 0.993912562258
Precision is : 0.927152317881


## SVM

In [18]:
from sklearn import svm

In [19]:
clf=svm.SVC()

In [ ]:
# TAKES A LONG TIME TO EXECUTE

param_dist = {'C': [0.1,10], 
              'kernel': ['linear','rbf', 'poly'],
              'gamma':[10,100]
              }
# run randomized search
n_iter_search = 4
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   scoring='roc_auc',
                                   random_state=2)
random_search.fit(x_train, y_train)
report(random_search.grid_scores_)

In [20]:
sv=svm.SVC(C=0.1, kernel='linear', gamma=10)
sv.fit(x_train,y_train)

predicted=sv.predict(x_test)
df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])
k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)
print('Precision is :', TP/(TP + FP))

predicted     0    1
real                
0          1784   23
1           101  123
123 1784 23 101
Accuracy is : 0.938946331856
Sensitivity is : 0.549107142857
Specificity is : 0.987271721085
Precision is : 0.842465753425


## Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
clf=GaussianNB()
clf.fit(x_train,y_train)

predicted=clf.predict(x_test)
df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])
k=pd.crosstab(df_test['real'],df_test["predicted"])
print(k)

TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

print(TP,TN,FP,FN)
print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)
print('Precision is :', TP/(TP + FP))

predicted     0    1
real                
0          1702  105
1            87  137
137 1702 105 87
Accuracy is : 0.905465288035
Sensitivity is : 0.611607142857
Specificity is : 0.941892639734
Precision is : 0.566115702479
